## Read Khan Academy Subject Page

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re

### Read Subject Page

In [18]:
# Read Khan Academy Subject Page
headers = {
   'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0',
   'X-Requested-With': 'XMLHttpRequest'
}
domain = 'https://www.khanacademy.org'
# source = domain + '/math/probability'
# source = domain + '/math/statistics-probability'
source = domain + '/math/ap-statistics'
response = requests.get(source, headers=headers)
response

# print(dir(response))

<Response [200]>

#### Read units

In [20]:
# Mount lessons array

# Lessons
lessons = []
_id = 0

# Get Page headings
unit_sequence = 0
# bsSubject = BeautifulSoup(response.text, 'lxml')
bsSubject = BeautifulSoup(response.text, 'html.parser')

field   = bsSubject.find('div', attrs={"aria-label": "breadcrumbs"}).get_text()
subject = bsSubject.find('h1',  attrs={"data-test-id": "unit-block-title"}).get_text()

# print(field)

for unit_block in bsSubject.findAll('div', class_='_12yy6f6l'):
   unit_sequence += 1


   # if unit_sequence == 3:
   #    break


   unit_title = unit_block.find('h3').get_text()
   print(unit_title)

   for group in unit_block.findAll('a', attrs={"data-test-id": "lesson-link"}):
      group_title = group.get('title')
      group_link  = group.get('href')
      
      # Load group page
      group_page = requests.get(domain + group_link, headers=headers)
      print("\t" + group_title)
      bsGroup = BeautifulSoup(group_page.text, 'html.parser')
      unit_group = bsGroup.find('ul', attrs={"data-test-id": "learnable-content-cells"})

      # Browse through lessons
      for lesson in unit_group.findAll('li'):
         lesson_title = lesson.get_text()
         lesson_link = lesson.find('a').get('href')

         # Get lesson type
         lesson_type = lesson.find(['span', 'svg'], attrs={"role": "img"}).get('aria-label')
         if lesson_type not in ['Article', 'Video']:
            lesson_type = 'Exercise'

         # Load lesson metadata
         # meta_data = getLessonMeta(domain + lesson_link, lesson_type)
         print("\t\t" + lesson_title + ' - ' + lesson_type)

         # Lesson content object
         _id += 1
         lesson = {
            '_id':           subject + ' - ' + str(_id).zfill(3),
            'subject':       subject,
            'unit':          unit_title,
            'unit_sequence': unit_sequence,
            'group':         group_title,
            'lesson':        lesson_title,
            'lesson_type':   lesson_type,
            'lesson_link':   lesson_link
         }
         lessons.append(lesson)


Unit 1: Exploring categorical data
	The language of variation: Variables
		Identifying individuals, variables and categorical variables in a data set - Video
		Individuals, variables, and categorical & quantitative data - Exercise
	Representing a categorical variable with graphs
		Creating a bar graph - Video
		Reading bar graphs: movies - Video
		Reading bar charts: comparing two sets of data - Video
		Read bar graphs - Exercise
	Representing two categorical variables
		Two-way frequency tables and Venn diagrams - Video
		Read two-way frequency tables - Exercise
		Create two-way frequency tables - Exercise
		Two-way relative frequency tables - Video
		Analyze two-way frequency tables - Exercise
		Interpreting two-way tables - Video
		Interpret two-way tables - Exercise
		Mosaic plots and segmented bar charts - Video
		Analyzing mosaic plots - Video
		Mosaic plots - Exercise
	Statistics for two categorical variables
		Marginal and conditional distributions - Video
		Identify marginal a

#### Read lessons and save them in the database

In [22]:
print(subject)

AP®︎/College Statistics


In [21]:
# print out an array or dictionary nicely
import json
print(json.dumps(lessons, indent=4))

[
    {
        "_id": "AP\u00ae\ufe0e/College Statistics - 001",
        "subject": "AP\u00ae\ufe0e/College Statistics",
        "unit": "Unit 1: Exploring categorical data",
        "unit_sequence": 1,
        "group": "The language of variation: Variables",
        "lesson": "Identifying individuals, variables and categorical variables in a data set",
        "lesson_type": "Video",
        "lesson_link": "/math/ap-statistics/analyzing-categorical-ap/xfb5d8e68:language-variation-variables/v/identifying-individuals-variables-and-categorical-variables-in-a-data-set"
    },
    {
        "_id": "AP\u00ae\ufe0e/College Statistics - 002",
        "subject": "AP\u00ae\ufe0e/College Statistics",
        "unit": "Unit 1: Exploring categorical data",
        "unit_sequence": 1,
        "group": "The language of variation: Variables",
        "lesson": "Individuals, variables, and categorical & quantitative data",
        "lesson_type": "Exercise",
        "lesson_link": "/math/ap-statistics/

In [23]:
# Save lessons in the database

# Create Khan Academy database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['khan-academy']

# Lessons collection
colLessons = db["lessons"]

query = { 'subject': subject }
x = colLessons.delete_many(query)
x = colLessons.insert_many(lessons)

#print list of the _id values of the inserted documents:
print(x.inserted_ids)

['AP®︎/College Statistics - 001', 'AP®︎/College Statistics - 002', 'AP®︎/College Statistics - 003', 'AP®︎/College Statistics - 004', 'AP®︎/College Statistics - 005', 'AP®︎/College Statistics - 006', 'AP®︎/College Statistics - 007', 'AP®︎/College Statistics - 008', 'AP®︎/College Statistics - 009', 'AP®︎/College Statistics - 010', 'AP®︎/College Statistics - 011', 'AP®︎/College Statistics - 012', 'AP®︎/College Statistics - 013', 'AP®︎/College Statistics - 014', 'AP®︎/College Statistics - 015', 'AP®︎/College Statistics - 016', 'AP®︎/College Statistics - 017', 'AP®︎/College Statistics - 018', 'AP®︎/College Statistics - 019', 'AP®︎/College Statistics - 020', 'AP®︎/College Statistics - 021', 'AP®︎/College Statistics - 022', 'AP®︎/College Statistics - 023', 'AP®︎/College Statistics - 024', 'AP®︎/College Statistics - 025', 'AP®︎/College Statistics - 026', 'AP®︎/College Statistics - 027', 'AP®︎/College Statistics - 028', 'AP®︎/College Statistics - 029', 'AP®︎/College Statistics - 030', 'AP®︎/Col